In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Awiros Work/Triple Riding/Colab Data.zip" -d "/content"

In [172]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam
from imutils import paths
import imagesize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import cv2
import os

In [173]:
class modelNet:
    @staticmethod
    def LeNet(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(4, (3, 3), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        model.add(Conv2D(8, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(16))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model
    
    @staticmethod
    def VGG16(classes):
        # initializing the model
        model = Sequential()
        
        
        model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Flatten())
        model.add(Dense(units=4096,activation="relu"))
        model.add(Dense(units=4096,activation="relu"))
        
        model.add(Dense(units=classes, activation="softmax"))
    

In [174]:
EPOCHS = 5
INIT_LR = 1e-3
BS = 8
CLASSES = 3
WIDTH = 224
HEIGHT = 224

In [175]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [176]:
imagePaths = sorted(list(paths.list_images("/content/Colab Data/trialDataset")))
random.seed(42)
random.shuffle(imagePaths)

### Taking out the images and training the model

In [177]:
for imagePath in imagePaths:    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (WIDTH, HEIGHT))
    image = img_to_array(image)
    data.append(image)
    

In [178]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{'double_person': 947, 'single_person': 963, 'triple_person': 612}

In [179]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [180]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)
del(data)
del(labels)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [181]:
print("[INFO] compiling model...")
model = modelNet.LeNet(width=WIDTH, height=HEIGHT, depth=3, classes=CLASSES)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [182]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 224, 224, 4)       112       
                                                                 
 activation_30 (Activation)  (None, 224, 224, 4)       0         
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 112, 112, 4)      0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 112, 112, 8)       808       
                                                                 
 activation_31 (Activation)  (None, 112, 112, 8)       0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 56, 56, 8)        0         
 g2D)                                                 

In [183]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=BS,
    validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...
Epoch 1/5
236/236 [==============================] - 3s 10ms/step - loss: 0.6843 - accuracy: 0.6801 - val_loss: 0.5683 - val_accuracy: 0.7544
Epoch 2/5
236/236 [==============================] - 2s 8ms/step - loss: 0.3473 - accuracy: 0.8598 - val_loss: 0.5487 - val_accuracy: 0.7496
Epoch 3/5
236/236 [==============================] - 2s 7ms/step - loss: 0.1749 - accuracy: 0.9474 - val_loss: 0.6262 - val_accuracy: 0.7829
Epoch 4/5
236/236 [==============================] - 2s 7ms/step - loss: 0.0667 - accuracy: 0.9851 - val_loss: 0.6408 - val_accuracy: 0.7829
Epoch 5/5
236/236 [==============================] - 2s 7ms/step - loss: 0.0238 - accuracy: 0.9968 - val_loss: 0.7876 - val_accuracy: 0.7845
[INFO] serializing network...


In [184]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Double and triple rider")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_multi_class_4.png")

In [185]:
predictions = model.predict(x=testX, batch_size=BS)

print(predictions.shape)

(631, 3)


In [186]:
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1),target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.76      0.70      0.73       251
single_person       0.75      0.86      0.80       239
triple_person       0.91      0.81      0.86       141

     accuracy                           0.78       631
    macro avg       0.81      0.79      0.79       631
 weighted avg       0.79      0.78      0.78       631



### Trying out test set with 3 classification

In [187]:
test_imagepaths = sorted(list(paths.list_images("/content/Colab Data/testSet")))
random.seed(42)
random.shuffle(test_imagepaths)

In [188]:
test_labels = []
test_data = []
for imagePath in test_imagepaths:    
    label = imagePath.split(os.path.sep)[-2]
    test_labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (WIDTH, HEIGHT))
    image = img_to_array(image)
    test_data.append(image)

test_labels = np.array(test_labels)
test_data = np.array(test_data, dtype="float") / 255.0


In [189]:
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)

In [190]:
test_predictions = model.predict(x=test_data, batch_size=32)

print(test_predictions.shape)

(221, 3)


In [191]:
test_predictions.argmax(axis=1)

array([0, 1, 0, 2, 0, 0, 1, 0, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 1, 0, 2, 2,
       0, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 2, 0, 0, 2, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2,
       1, 0, 0, 2, 1, 1, 1, 0, 0, 0, 1, 1, 1, 2, 2, 0, 0, 2, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 2, 0, 2, 1, 1, 2, 2, 1, 0, 0, 2, 2, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2,
       1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0,
       0])

In [192]:
print(classification_report(test_labels.argmax(axis=1),test_predictions.argmax(axis=1),target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.29      0.41      0.34        78
single_person       0.63      0.61      0.62        80
triple_person       0.25      0.13      0.17        63

     accuracy                           0.40       221
    macro avg       0.39      0.38      0.38       221
 weighted avg       0.40      0.40      0.39       221

